In [1]:
!pip install transformers torch sentence-transformers faiss-cpu flask pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 37.9 MB/s eta 0:00:00


In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
from flask import Flask, request, jsonify
from pyngrok import ngrok
import yaml


In [16]:
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": """To create a Shoplite account, users visit the registration page and provide their email address, a strong password, and basic profile information such as full name and phone number. Shoplite supports two primary account types: buyer accounts (free for consumers) and seller accounts (for merchants). Seller accounts require additional business verification including business name, tax ID or VAT number, and a business address. After submission, Shoplite sends an email verification link that must be clicked within 24 hours to activate the account.

For seller onboarding, the platform will also request a short description of the business, a clear logo image, and bank details for payments. Shoplite's verification process typically takes 2-3 business days; during this time the seller account is created in 'Pending Verification' mode. Sellers can preview their storefront but cannot publish products until verification is complete. Users may connect OAuth providers for faster sign-in (Google, Apple, Facebook). Password security requires at least 8 characters, one uppercase, one lowercase, and one digit. Two-factor authentication (2FA) can be enabled in account settings using SMS or an authenticator app for added security.

Account management features include password reset via emailed tokens, profile editing, account deletion requests, and a dashboard for order history and saved payment methods. GDPR-compliant data export and portability options are available; users can request a JSON export of their personal data from the account settings page. If a user reports suspicious activity, Shoplite's support team can temporarily suspend the account while the issue is investigated."""
    },
    {
        "id": "doc2",
        "title": "Product Search and Filtering Features",
        "content": """Shoplite provides a fast product search powered by keyword matching plus semantic search signals for better relevance. The search bar supports autocomplete suggestions, typo tolerance, and suggested categories. Search results can be filtered by category, price range, seller rating, availability (in-stock), and shipping options (free shipping, fast delivery). Users may sort results by relevance, price (low-to-high / high-to-low), newest arrivals, and top-rated.

Advanced filtering supports attribute-based faceted navigation for products that expose attributes (size, color, material). Filters are combined using AND logic within attribute groups and AND across groups for predictable narrowing. Facets display result counts so users understand the scope of narrowing. The search backend indexes product title, description, SKU, tags, categories, and seller-provided metadata. Sellers may add boost weights for promoted attributes via the seller console to highlight items for specific queries.

Shoplite also supports saved searches and email alerts for price drops or new matches. For international catalogs, the search automatically maps currency conversion and language-specific tokenization to improve relevance. The product detail page highlights similar items and cross-sells using collaborative filtering and content-based similarity; this improves discoverability and cart add rates."""
    },
    {
        "id": "doc3",
        "title": "Shopping Cart and Checkout Process",
        "content": """The Shoplite shopping cart supports multiple sellers in the same cart, consolidated checkout, and per-seller order splitting where required by seller policies. Cart contents persist across sessions for logged-in users and can be synchronized across devices. Users can save items for later, apply promotional codes, and view estimated taxes and shipping before placing orders.

Checkout is a multi-step, clearly-labeled process: (1) Shipping Address selection or entry, (2) Shipping Method selection with estimated delivery and costs, (3) Payment method selection, and (4) Order review with final confirmation. Guest checkout is allowed but limited: guests must provide an email for order confirmation and tracking. For buyers with saved addresses, Express Checkout pre-fills fields to speed the flow. Users receive an order confirmation email upon successful payment and a separate shipping notification when the seller marks items as shipped.

Cart-level safeguards include inventory re-check at checkout time (to avoid overselling), automatic coupon stacking rules (to prevent abuse), and a timeout-based reserved inventory hold for high-demand or limited-stock items. For split shipments (items from different sellers), Shoplite displays estimated separate shipping times and combined invoice details in the order summary."""
    },
    {
        "id": "doc4",
        "title": "Payment Methods and Security",
        "content": """Shoplite supports multiple payment methods including major credit/debit cards (Visa, MasterCard, American Express), digital wallets (Apple Pay, Google Pay), and integration with payment gateways (Stripe, PayPal, Adyen). For seller payouts the platform supports direct bank transfer and integrations with payout partners depending on the country. Payment information is tokenized and never stored in raw form on Shoplite's servers; tokenization is handled by the payment processor to minimize PCI scope.

Security measures include TLS (HTTPS) for all pages that handle sensitive data, strict Content Security Policy (CSP) headers, and encryption of payment metadata at rest. Cardholder data transmission follows PCI-DSS requirements; Shoplite uses a certified payment processor to handle actual card data. Fraud detection leverages rule-based checks (velocity, IP-risk scoring) and machine learning models for fraudulent pattern detection. High-risk orders may be flagged for manual review by a dedicated team.

For recurring billing and subscriptions, Shoplite stores a processor-managed token representing the payment method. Users can manage saved payment methods from the account settings page and revoke tokens at any time. Refunds are processed back to the original payment method subject to gateway policies."""
    },
    {
        "id": "doc5",
        "title": "Order Tracking and Delivery",
        "content": """After a seller marks an order as shipped, Shoplite generates tracking metadata and sends an email and in-app notification to the buyer. The platform supports carrier integrations (UPS, FedEx, DHL, local couriers) to fetch status updates automatically and present a unified tracking timeline in the order details page. Estimated delivery windows are shown when the seller chooses a shipping method; these estimates are based on carrier SLA plus handling time.

For items shipped in multiple packages or from multiple sellers, Shoplite merges the tracking information into a single order view so buyers can monitor each package. Buyers may request delivery hold instructions or changes through the carrier directly if supported; Shoplite surfaces carrier links and contact information.

International shipments show customs notes, expected duties and taxes (estimated), and possible additional delays. Delivery exceptions (lost, delayed, damaged) are escalated to customer support with a reference ticket number. Sellers must upload proof of shipment or carrier receipts in disputed cases. Shoplite's support team mediates when buyers file claims under the platform protection policy."""
    },
    {
        "id": "doc6",
        "title": "Return and Refund Policies",
        "content": """Shoplite's general return policy provides a 30-day return window for most non-perishable items measured from the delivery date, unless otherwise specified on the product page. Prime-exempt categories (digital goods, perishable items, personalized products) may have different return rules. Buyers must request a return via their order page which generates a return authorization code (RA). The RA is required to process the return and to prevent unauthorized returns.

Refunds are processed once the returned item is received and inspected by the seller or after the seller accepts the return via the platform. Refunds issued via the original payment method typically take 5-10 business days depending on the payment processor. Shoplite provides a 'seller-protection' and 'buyer-protection' program which acts as an escrow for dispute resolution when necessary.

Return shipping responsibilities depend on the reason for return: if the item is defective or not-as-described, the seller covers return shipping; if the buyer changed their mind, the buyer may be responsible for return postage. Sellers may offer prepaid return labels for convenience. Re-stocking fees, if applicable, must be disclosed on the product page."""
    },
    {
        "id": "doc7",
        "title": "Product Reviews and Ratings",
        "content": """Shoplite allows verified buyers to submit product reviews and provide star ratings (1-5). Reviews are linked to the order ID to ensure reviewer verification. Shoplite supports text reviews, photo uploads, and seller responses. To promote trust, the platform displays 'verified purchase' badges next to reviews originating from completed orders.

Moderation rules prohibit hate speech, profanity, and defamation. Review moderation is a mixture of automated filters (to detect spam, repetitive content) and human review for flagged items. Sellers may not offer incentives for positive reviews; any incentivized review must be disclosed and follow local regulations.

Aggregated product ratings are calculated as a weighted average, giving slightly higher weight to recent reviews to reflect current product quality. Review analytics are available to sellers to identify product issues, common complaints, and feature requests. Buyers can sort reviews by most helpful, most recent, or highest rated."""
    },
    {
        "id": "doc8",
        "title": "Seller Account Setup and Management",
        "content": """Sellers register by selecting the seller account option and completing a business profile with contact details, tax information, and bank payout instructions. The seller dashboard organizes product listings, order management, inventory controls, promotions, and performance analytics. During onboarding, sellers accept the marketplace agreement which outlines fees, dispute resolution, and acceptable use policies.

The dashboard provides a product creation wizard that guides sellers through uploading images, UPC/SKU management, category selection, and shipping templates. Shipping templates allow sellers to define zone-based rates, free-shipping thresholds, and handling times. Sellers can also enable return rules and provide custom policies that are displayed to buyers.

Account health metrics highlight late shipments, cancellation rates, and dispute rates; repeated policy violations may lead to temporary suspension. Sellers may connect to the Shoplite API for bulk product uploads and integrations with common ERPs or inventory systems. Role-based access within a seller account allows multiple users with different permissions (manager, fulfillment, accounting)."""
    },
    {
        "id": "doc9",
        "title": "Inventory Management for Sellers",
        "content": """Shoplite includes an inventory management system that supports single-SKU and multi-SKU catalogs, stock thresholds, and low-inventory alerts. Sellers can manage inventory manually through the dashboard or integrate via API/webhooks for real-time sync from their warehouses or third-party fulfillment partners.

Key features include batch updates for pricing and stock, CSV import/export for bulk changes, and reserved inventory mechanisms to hold stock during checkout for a short duration. Inventory auditing tools track adjustments, reasons (returns, stock counts, damage), and user actions for traceability. Sellers can optionally enable backorder handling with clear buyer-facing messaging about expected delays.

For sellers using fulfillment partners, Shoplite supports FBA-like flows where a fulfillment provider updates stock levels via API. Threshold rules can trigger automatic purchase orders or alert the seller to restock popular SKUs. Inventory reports can be exported for accounting and forecasting."""
    },
    {
        "id": "doc10",
        "title": "Commission and Fee Structure",
        "content": """Shoplite charges a commission on each sale which varies by category and seller tier. Commission is calculated as a percentage of the item price plus a fixed marketplace fee. Sellers subscribe to different tiers (Standard, Pro, Enterprise) that offer varying fee structures, marketing benefits, and reduced commission rates for higher volumes.

Other fees include payment processing fees charged by the payment gateway, optional promoted listing fees for boosting visibility, and chargeback fees when disputes are resolved against the seller. Payout schedules differ by country but commonly run on a 7-day rolling basis after order delivery confirmation to reduce fraud exposure. Sellers can upgrade to faster payouts for a small fee.

Fee breakdown details are provided in the seller dashboard and on the seller agreement. Shoplite provides monthly statements showing commission deductions, refunds, promoter fees, and net payouts."""
    },
    {
        "id": "doc11",
        "title": "Customer Support Procedures",
        "content": """Shoplite provides multiple support channels: in-app messaging, email support, phone support (for priority sellers), and an extensive help center. Customer support follows a tiered SLA model: basic inquiries are answered within 48 hours, priority inquiries within 24 hours, and critical issues (safety, fraud) within 6 hours.

The in-app support feature routes messages to the right team based on topic (orders, returns, technical). Support agents use a CRM with context windows showing recent orders, payment status, and chat history. Automated response templates handle common issues (tracking questions, refund timelines), while complex disputes escalate to human agents. Agents can issue temporary refunds, request proof of shipment from sellers, or open investigations with carriers.

For international support, Shoplite offers multi-lingual agents and localized help articles. A knowledge base articles system allows agents to link authoritative help pages directly in replies."""
    },
    {
        "id": "doc12",
        "title": "Mobile App Features",
        "content": """Shoplite's mobile app (iOS and Android) offers a streamlined shopping experience: personalized home feed, push notifications for deals and order updates, barcode scanning for quick product lookup, and one-tap checkout with saved payment methods. The app supports deep linking to product pages, seller storefronts, and in-app messaging.

Sellers have a companion Seller App for quick order processing, photo-based product uploads, and inventory adjustments. The app supports offline photo capture and draft listings that can be completed when online. The buyer app includes gestures for swiping product carousels, wishlists, and integration with device-native payments (Apple Pay / Google Pay).

Security features on mobile include biometric authentication (Face ID / Touch ID) for faster sign-in and secure token storage managed by the OS. Push notifications are configurable by type."""
    },
    {
        "id": "doc13",
        "title": "API Documentation for Developers",
        "content": """Shoplite provides a RESTful API for developers covering product catalog, order management, inventory sync, and webhook notifications. The API uses OAuth 2.0 for authorization and JSON over HTTPS for payloads. Endpoints follow predictable resource-based patterns: /products, /orders, /sellers, /inventory. Rate limits are documented per tier and enforcement is via standard HTTP status codes (429 Too Many Requests when exceeded).

Webhook support enables push notifications to merchant systems for order creation, fulfillment updates, returns, and payment events. Developers can subscribe to webhooks and provide a retry URL to handle transient delivery failures. API client libraries are available in Python and Node.js with pagination helpers and retry logic.

Sandbox environments allow testing of webhooks and payments without processing real money. API keys and scopes are managed from the developer console, and rotating keys is supported."""
    },
    {
        "id": "doc14",
        "title": "Security and Privacy Policies",
        "content": """Shoplite follows industry-standard security practices including HTTPS everywhere, regular penetration testing, vulnerability disclosure program, and least-privilege access controls. Personal data handling complies with GDPR and other regional privacy laws; users can request data exports or deletion. The platform logs access to sensitive operations and keeps audit trails.

Access to production systems is limited via VPN and multi-factor authentication for staff. Sensitive credentials (DB passwords, API keys) are stored in a secrets manager and rotated regularly. Privacy policies describe data usage: order details for fulfillment, analytics for product improvement (aggregated and anonymized), and marketing opt-ins. Shoplite does not sell user data to third parties."""
    },
    {
        "id": "doc15",
        "title": "Promotional Codes and Discounts",
        "content": """Shoplite supports promotional codes (percentage, fixed-amount, free-shipping) with start/end dates, usage limits, and eligibility rules (new customers only, specific categories). Sellers and the marketplace team can create promotions. Coupons can be stackable or mutually exclusive depending on business rules. Promotion analytics show redemption rates and revenue impact.

Discounts apply at cart or item level; the checkout engine applies promotions deterministically based on priority and stacking rules. Fraud controls limit abusive coupon behavior (mass redemptions, multiple accounts) using rate limits and account linking heuristics. For promotional experiments, Shoplite supports A/B tests to evaluate financial impact before wide rollout."""
    }
]

print(f'Loaded {len(KNOWLEDGE_BASE)} documents in the notebook knowledge base')



Loaded 15 documents in the notebook knowledge base


In [18]:

PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible"
        ],
        "response_format": "Answer: [Your concise response based on context]\nSources: [List document titles referenced]"
    },
    "multi_source_synthesis": {
        "role": "You are a Shoplite knowledge synthesis assistant.",
        "goal": "Combine information from multiple documents to provide a clear, step-by-step answer.",
        "context_guidelines": [
            "Prioritize authoritative policy documents for policy-related questions",
            "When combining documents, name each source for transparency"
        ],
        "response_format": "Summary: [Short synthesis]\nSteps: [Numbered action steps]\nSources: [List document titles referenced]"
    },
    "no_context_refuse": {
        "role": "You are a Shoplite assistant that refuses when no context exists.",
        "goal": "If the knowledge base contains no relevant information, politely refuse and offer alternatives.",
        "context_guidelines": [
            "Do not hallucinate facts",
            "Suggest where the user can find help or ask for clarification"
        ],
        "response_format": "Answer: I do not have enough information in the Shoplite docs to answer that safely.\nSuggestion: [What the user can try or which doc to consult]"
    },
    "clarification_prompt": {
        "role": "You are a Shoplite assistant that requests clarifying information when questions are ambiguous.",
        "goal": "Prompt the user for specifics needed to find the right documents or produce a correct answer.",
        "context_guidelines": [
            "Ask for only the minimum required detail",
            "Provide examples to guide the user"
        ],
        "response_format": "Clarifying question: [Single focused question]"
    }
}

print("Prompts dictionary loaded successfully!")



Prompts dictionary loaded successfully!


In [5]:
# Install dependencies (run this once at the top)
!pip install transformers torch accelerate sentence-transformers safetensors

# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [8]:
from huggingface_hub import login

# Paste your token in the input prompt
login()


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Use an open model instead of Llama 3 gated model
model_name = "NousResearch/Llama-2-7b-hf"  # ✅ Open source

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [12]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
doc_texts = [doc['content'] for doc in KNOWLEDGE_BASE]
doc_embeddings = embed_model.encode(doc_texts, convert_to_numpy=True)

# Build FAISS index
dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(doc_embeddings)


NameError: name 'SentenceTransformer' is not defined

In [13]:
!pip install sentence-transformers



In [14]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings from your knowledge base
doc_texts = [doc['content'] for doc in KNOWLEDGE_BASE]
doc_embeddings = embed_model.encode(doc_texts, convert_to_numpy=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

TypeError: 'set' object is not subscriptable

In [17]:
doc_texts = [doc['content'] for doc in KNOWLEDGE_BASE]
doc_embeddings = embed_model.encode(doc_texts, convert_to_numpy=True)


In [19]:
!pip install sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 33.9 MB/s eta 0:00:00


In [20]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


In [21]:
# We use a lightweight embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')


In [22]:
# Extract text content from your KNOWLEDGE_BASE
doc_texts = [doc['content'] for doc in KNOWLEDGE_BASE]
doc_ids = [doc['id'] for doc in KNOWLEDGE_BASE]


In [23]:
# Convert document texts into embeddings
doc_embeddings = embed_model.encode(doc_texts, convert_to_numpy=True)
print(f"Generated embeddings for {len(doc_embeddings)} documents")


Generated embeddings for 15 documents


In [24]:
# Dimension of embeddings
embedding_dim = doc_embeddings.shape[1]

# Initialize FAISS index
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(doc_embeddings)

print(f"FAISS index created with {index.ntotal} vectors")


FAISS index created with 15 vectors


In [25]:
def retrieve_documents(query, top_k=3):
    # Convert query to embedding
    query_embedding = embed_model.encode([query], convert_to_numpy=True)

    # Search FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the actual document texts
    results = [KNOWLEDGE_BASE[i] for i in indices[0]]
    return results


In [26]:
def format_context(retrieved_docs):
    context = ""
    for doc in retrieved_docs:
        context += f"Title: {doc['title']}\nContent: {doc['content']}\n\n"
    return context


In [27]:
def generate_prompt(query, prompt_type="base_retrieval_prompt"):
    retrieved_docs = retrieve_documents(query, top_k=3)
    context = format_context(retrieved_docs)

    prompt_template = PROMPTS[prompt_type]

    prompt_text = (
        f"Role: {prompt_template['role']}\n"
        f"Goal: {prompt_template['goal']}\n"
        f"Context Guidelines: {'; '.join(prompt_template['context_guidelines'])}\n"
        f"Context: {context}\n"
        f"User Question: {query}\n"
        f"{prompt_template['response_format']}"
    )
    return prompt_text


In [28]:
user_query = "How do I create a seller account on Shoplite?"
prompt = generate_prompt(user_query)
print(prompt)


Role: You are a helpful Shoplite customer service assistant.
Goal: Provide accurate answers using only the provided Shoplite documentation.
Context Guidelines: Use only information from the provided document snippets; Cite specific documents when possible
Context: Title: Shoplite User Registration Process
Content: To create a Shoplite account, users visit the registration page and provide their email address, a strong password, and basic profile information such as full name and phone number. Shoplite supports two primary account types: buyer accounts (free for consumers) and seller accounts (for merchants). Seller accounts require additional business verification including business name, tax ID or VAT number, and a business address. After submission, Shoplite sends an email verification link that must be clicked within 24 hours to activate the account.

For seller onboarding, the platform will also request a short description of the business, a clear logo image, and bank details for p

In [29]:
while True:
    user_query = input("You: ")
    if user_query.lower() in ["exit", "quit"]:
        print("Exiting chat...")
        break

    # Generate the prompt with retrieval
    prompt = generate_prompt(user_query)

    # Here you would call your LLM if you want generated answers
    # For now, we just print the prompt
    print("\n--- Generated Prompt ---")
    print(prompt)
    print("------------------------\n")


You: How do I create a seller account?

--- Generated Prompt ---
Role: You are a helpful Shoplite customer service assistant.
Goal: Provide accurate answers using only the provided Shoplite documentation.
Context Guidelines: Use only information from the provided document snippets; Cite specific documents when possible
Context: Title: Shoplite User Registration Process
Content: To create a Shoplite account, users visit the registration page and provide their email address, a strong password, and basic profile information such as full name and phone number. Shoplite supports two primary account types: buyer accounts (free for consumers) and seller accounts (for merchants). Seller accounts require additional business verification including business name, tax ID or VAT number, and a business address. After submission, Shoplite sends an email verification link that must be clicked within 24 hours to activate the account.

For seller onboarding, the platform will also request a short descri